Instituto Infnet

Disciplina Análise de dados com Spark

Aluno: Victor Luiz de Oliveira e Silva

In [1]:
!pip install pyspark
import pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=48bccb5ba727afdbb27ef8dfc32a3d98fabd259892d0f9b990edf081424cd912
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
## Você só vai precisar dessa parte se fizer no Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Importando bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max, when, isnan, min, avg

In [4]:
#Criando sessão local no spark
spark = SparkSession.builder.appName("Exercicio").master('local[*]').getOrCreate()

In [5]:
#Carga primeiro de dois csv de dados de preço de combustível no Brasil
df_1_semestre = spark.read.csv(path='drive/MyDrive/Arquivos/ca-2020-01.csv',header=True,inferSchema=True,sep = ';')

In [6]:

df_1_semestre.show(5)

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-------------------+----------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|        Complemento|    Bairro|      Cep|   Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-------------------+----------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP|GUARULHOS|AUTO POSTO SAKAMO...| 49.051.667/0001-02|RODOVIA PRESIDENT...|       S/N|KM 210,5-SENT SP/RJ|BONSUCESSO|07178-580|  GASOLINA|    03/01/2020|         4,399|           NULL|       R$ / litro|PETROBRAS DIST

In [7]:
#Carga do segundo csv de preço de combustivel no Brasil
df_2_semestre = spark.read.csv(path='drive/MyDrive/Arquivos/ca-2020-02.csv',header=True,inferSchema=True,sep = ';')

In [8]:
df_2_semestre.show(5)

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|Complemento|        Bairro|      Cep|   Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|       306|       NULL|JARDIM ZULMIRA|18061-000|  GASOLINA|    01/07/2020|         3,559|          3,253|       R$ / litro|              BRANCA|
|       

In [9]:
#Acrescentando os 2 dataframes um embaixo do outro.
df_precos_2020 = df_1_semestre.union(df_2_semestre)
df_precos_2020.show()

+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+-------------------+------------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|      Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|        Complemento|      Bairro|      Cep|   Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+-------------------+------------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP|      GUARULHOS|AUTO POSTO SAKAMO...| 49.051.667/0001-02|RODOVIA PRESIDENT...|       S/N|KM 210,5-SENT SP/RJ|  BONSUCESSO|07178-580|  GASOLINA|    03/01/2020|         4,399|           NULL|

In [10]:
df_1_semestre.count()


496663

In [11]:
df_2_semestre.count()

222637

In [12]:
#Conferindo o tamanho do dataset
df_precos_2020.count()

719300

In [13]:
#Alterando o nome das colunas que tem espaço e usando _ no lugar
current_columns = df_precos_2020.columns # Returns list of columns as python list
new_columns = list(map(lambda item : item.replace(" ","_").upper(),current_columns))


In [14]:
#Alterando os nomes pelo nome new_columns com _ no lugar de espaços para facilitar nas chamadas de funçoes
from functools import reduce
final_df = reduce(lambda data, idx: data.withColumnRenamed(current_columns[idx], new_columns[idx]), range(len(current_columns)), df_precos_2020)

final_df.printSchema()

root
 |-- REGIAO_-_SIGLA: string (nullable = true)
 |-- ESTADO_-_SIGLA: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- REVENDA: string (nullable = true)
 |-- CNPJ_DA_REVENDA: string (nullable = true)
 |-- NOME_DA_RUA: string (nullable = true)
 |-- NUMERO_RUA: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- DATA_DA_COLETA: string (nullable = true)
 |-- VALOR_DE_VENDA: string (nullable = true)
 |-- VALOR_DE_COMPRA: string (nullable = true)
 |-- UNIDADE_DE_MEDIDA: string (nullable = true)
 |-- BANDEIRA: string (nullable = true)



In [15]:
#Substituição de vírgulas por ponto nos valores numéricos para conversão do tipo string para float
from pyspark.sql.functions import translate
final_df = final_df.withColumn('VALOR_DE_VENDA', translate('VALOR_DE_VENDA', ',', '.'))

In [16]:
#Alteração para float da coluna VALOR_DE_VENDA
from pyspark.sql.types import FloatType,DateType

final_df2 = final_df.withColumn("VALOR_DE_VENDA" ,final_df["VALOR_DE_VENDA"].cast(FloatType()))
            #        .withColumn("DATA_DA_COLETA",final_df["DATA_DA_COLETA"].cast(DateType()))

final_df2.printSchema()

root
 |-- REGIAO_-_SIGLA: string (nullable = true)
 |-- ESTADO_-_SIGLA: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- REVENDA: string (nullable = true)
 |-- CNPJ_DA_REVENDA: string (nullable = true)
 |-- NOME_DA_RUA: string (nullable = true)
 |-- NUMERO_RUA: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- DATA_DA_COLETA: string (nullable = true)
 |-- VALOR_DE_VENDA: float (nullable = true)
 |-- VALOR_DE_COMPRA: string (nullable = true)
 |-- UNIDADE_DE_MEDIDA: string (nullable = true)
 |-- BANDEIRA: string (nullable = true)



In [17]:
final_df2.show(30)

+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+-------------------+--------------------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|REGIAO_-_SIGLA|ESTADO_-_SIGLA|      MUNICIPIO|             REVENDA|    CNPJ_DA_REVENDA|         NOME_DA_RUA|NUMERO_RUA|        COMPLEMENTO|              BAIRRO|      CEP|   PRODUTO|DATA_DA_COLETA|VALOR_DE_VENDA|VALOR_DE_COMPRA|UNIDADE_DE_MEDIDA|            BANDEIRA|
+--------------+--------------+---------------+--------------------+-------------------+--------------------+----------+-------------------+--------------------+---------+----------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP|      GUARULHOS|AUTO POSTO SAKAMO...| 49.051.667/0001-02|RODOVIA PRESIDENT...|       S/N|KM 210,5-SENT SP/RJ|          BONSUCESSO|07178-580|  GASOLINA|    03/01/2020

In [18]:
#Média de valor de venda da gasolina no inicio do ano de 2020
final_df2.groupBy('DATA_DA_COLETA','PRODUTO')\
          .avg('VALOR_DE_VENDA')\
          .select('avg(VALOR_DE_VENDA)','DATA_DA_COLETA','PRODUTO')\
          .filter(col('PRODUTO')=="GASOLINA")\
          .orderBy('DATA_DA_COLETA').show(1)


+-------------------+--------------+--------+
|avg(VALOR_DE_VENDA)|DATA_DA_COLETA| PRODUTO|
+-------------------+--------------+--------+
|  4.596277451640025|    01/01/2020|GASOLINA|
+-------------------+--------------+--------+
only showing top 1 row



In [19]:
#Média do valor de venda da gasolina no final do ano de 2020
final_df2.groupBy('DATA_DA_COLETA','PRODUTO')\
          .avg('VALOR_DE_VENDA')\
          .select('avg(VALOR_DE_VENDA)','DATA_DA_COLETA','PRODUTO')\
          .sort(col('DATA_DA_COLETA').desc())\
          .filter(col('DATA_DA_COLETA')=="31/12/2020")\
          .filter(col('PRODUTO')=="GASOLINA")\
          .show(1)


+-------------------+--------------+--------+
|avg(VALOR_DE_VENDA)|DATA_DA_COLETA| PRODUTO|
+-------------------+--------------+--------+
|  4.410865641351956|    31/12/2020|GASOLINA|
+-------------------+--------------+--------+



In [20]:
#Dados de preço de combustivel dos EUA
df_preco_usa = spark.read.csv(path='drive/MyDrive/Arquivos/PET_PRI_GND_DCUS_NUS_W.csv',header=True,inferSchema=True,sep = ',')
df_preco_usa = df_preco_usa.toDF('DATA', 'TODOS_TIPOS_FORMULA', 'TODOS_TIPOS_CONVENCIONAL', 'TODOS_TIPOS_REFORMULADA', 'REGULAR_TODOS_TIPOS','REGULAR_CONVENCIONAL',
        'REGULAR_REFORMULADA','MEDIA_TODAS_FORMULAS', 'MEDIA_CONVENCIONAL', 'MEDIA_REFORMULADA', 'PREMIUM_TODAS_FORMULAS', 'PREMIUM_CONVENCIONAL',
        'PREMIUM_REFORMULADA' , 'DIESEL')

In [21]:
#Primerias 30 linhas dos diferentes tipos de gasolina vendidos nos EUA (em dólares por galão)
df_preco_usa.show(30)

+----------+-------------------+------------------------+-----------------------+-------------------+--------------------+-------------------+--------------------+------------------+-----------------+----------------------+--------------------+-------------------+------+
|      DATA|TODOS_TIPOS_FORMULA|TODOS_TIPOS_CONVENCIONAL|TODOS_TIPOS_REFORMULADA|REGULAR_TODOS_TIPOS|REGULAR_CONVENCIONAL|REGULAR_REFORMULADA|MEDIA_TODAS_FORMULAS|MEDIA_CONVENCIONAL|MEDIA_REFORMULADA|PREMIUM_TODAS_FORMULAS|PREMIUM_CONVENCIONAL|PREMIUM_REFORMULADA|DIESEL|
+----------+-------------------+------------------------+-----------------------+-------------------+--------------------+-------------------+--------------------+------------------+-----------------+----------------------+--------------------+-------------------+------+
|01/02/1995|              1.127|                   1.104|                  1.231|              1.079|               1.063|              1.167|                1.17|             1.159|  

In [22]:
df_preco_usa.printSchema()

root
 |-- DATA: string (nullable = true)
 |-- TODOS_TIPOS_FORMULA: double (nullable = true)
 |-- TODOS_TIPOS_CONVENCIONAL: double (nullable = true)
 |-- TODOS_TIPOS_REFORMULADA: double (nullable = true)
 |-- REGULAR_TODOS_TIPOS: double (nullable = true)
 |-- REGULAR_CONVENCIONAL: double (nullable = true)
 |-- REGULAR_REFORMULADA: double (nullable = true)
 |-- MEDIA_TODAS_FORMULAS: double (nullable = true)
 |-- MEDIA_CONVENCIONAL: double (nullable = true)
 |-- MEDIA_REFORMULADA: double (nullable = true)
 |-- PREMIUM_TODAS_FORMULAS: double (nullable = true)
 |-- PREMIUM_CONVENCIONAL: double (nullable = true)
 |-- PREMIUM_REFORMULADA: double (nullable = true)
 |-- DIESEL: double (nullable = true)



In [23]:
#Criar dataframe com o preço da gasolina nos eua em dolares por galão
preco_inicio_ano_usa = df_preco_usa.groupBy('DATA','MEDIA_CONVENCIONAL')\
          .avg('MEDIA_CONVENCIONAL')\
          .select('avg(MEDIA_CONVENCIONAL)','DATA')\
          .sort(col('DATA').desc())\
          .filter(col('DATA')=="01/06/2020")\
          .withColumnRenamed('avg(MEDIA_CONVENCIONAL)', 'MEDIA_CONVENCIONAL') \
        #  .show(1)

In [24]:
# Dataframe do preço da gasolina no inicio do ano de 2020 em dólares por galão
preco_inicio_ano_usa.show(1)

+------------------+----------+
|MEDIA_CONVENCIONAL|      DATA|
+------------------+----------+
|             2.833|01/06/2020|
+------------------+----------+



In [25]:
preco_inicio_ano_usa.head().MEDIA_CONVENCIONAL * 4.05 /3.79
# Media de preço de gasolina por galão no inicio do ano de 2020 multiplicada pelo valor do dólar na época dividida pela quantidade de litros por galão


3.0273482849604223

In [26]:
#Descobrindo preço da gasolina no fim do ano de 2020 e criando um dataframe com esse dado
preco_fim_ano_usa = df_preco_usa.groupBy('DATA','MEDIA_CONVENCIONAL')\
          .avg('MEDIA_CONVENCIONAL')\
          .select('avg(MEDIA_CONVENCIONAL)','DATA')\
          .sort(col('DATA').desc())\
          .filter(col('DATA')=="12/28/2020")\
          .withColumnRenamed('avg(MEDIA_CONVENCIONAL)', 'MEDIA_CONVENCIONAL')

In [27]:
# Media de preço de gasolina por galão no fim do ano de 2020 multiplicada pelo valor do dólar na época dividida pela quantidade de litros por galão

preco_fim_ano_usa.head().MEDIA_CONVENCIONAL * 5.23 /3.79


3.4250290237467023

In [28]:
#Descobrir o preço máximo da gasolina no Brasil em reais por litro no ano de 2020
final_df2.groupBy('PRODUTO')\
          .max('VALOR_DE_VENDA')\
          .select('max(VALOR_DE_VENDA)','PRODUTO')\
          .filter(col('PRODUTO')=="GASOLINA")\
          .show()


+-------------------+--------+
|max(VALOR_DE_VENDA)| PRODUTO|
+-------------------+--------+
|              5.899|GASOLINA|
+-------------------+--------+



In [29]:
#Preço máximo da gasolina em reais por litro após a conversão de doláres por galão no ano de 2020
df_preco_usa.createOrReplaceTempView("TAB")
spark.sql("select max(MEDIA_CONVENCIONAL) from TAB where DATA LIKE '%2020%' ").head()[0]  * 5.23/3.79

3.9093905013192614

In [30]:
#Preço máximo do diesel em ambos os datasets
spark.sql("select max(DIESEL) from TAB where DATA LIKE '%2020%' ").head()[0]

3.079

In [31]:
preco_max_diesel = spark.sql("select max(DATA) as data, MAX(DIESEL) as diesel from TAB WHERE DATA LIKE '%2020%' group by diesel  HAVING DIESEL = MAX(DIESEL) ").orderBy('data')#.show(1)
spark.sql("select max(DATA) as data, MAX(DIESEL) as diesel from TAB WHERE DATA LIKE '%2020%' group by diesel  HAVING DIESEL = MAX(DIESEL) ").orderBy('data').show(1)

+----------+------+
|      data|diesel|
+----------+------+
|01/06/2020| 3.079|
+----------+------+
only showing top 1 row



In [32]:
#Multiplicando pelo dolar em 1º de junho
preco_max_diesel.head().diesel * 4.05 / 3.79

3.2902242744063326

In [33]:
#Descobrir o preço máximo do diesel no Brasil em reais por litro no ano de 2020
final_df2.groupBy('PRODUTO','DATA_DA_COLETA')\
          .max('VALOR_DE_VENDA')\
          .select('max(VALOR_DE_VENDA)','PRODUTO','DATA_DA_COLETA')\
          .filter(col('PRODUTO')=="DIESEL")\
          .show(1)

+-------------------+-------+--------------+
|max(VALOR_DE_VENDA)|PRODUTO|DATA_DA_COLETA|
+-------------------+-------+--------------+
|               4.99| DIESEL|    07/01/2020|
+-------------------+-------+--------------+
only showing top 1 row

